In [370]:
import pandas as pd
import numpy as np
import re

In [380]:
all_sheets = pd.read_excel("../Final Dashboard Data- 17-25 (1).xlsx", sheet_name=None)

for name, df in all_sheets.items():
    print(name, df.shape)

Dadar (11, 60)
Cuff Parade (5, 16)
Chimbai Beach, Bandra (5, 16)
Worli Fort, Worli (5, 18)
Madh, Malad (5, 16)
Erangal, Malad (5, 16)
Gorai, Bhayander (4, 16)
Prabhadevi Beach (8, 16)
Silver Beach (4, 15)
Juhu Koliwada, Juhu (10, 16)
Sheet1 (8, 3)


In [381]:
def clean_period_year(series):
    cleaned = (
        series.astype(str)
              .str.strip()                               # strip outer spaces
              .str.replace(r"\s*-\s*", "-", regex=True)  # normalize spaces around '-'
    )

    # Normalize `(Legrand)` suffix
    cleaned = cleaned.str.replace(r"\(\s*legrand\s*\)", "(Legrand)", 
                                  flags=re.IGNORECASE, regex=True)

    return cleaned

def clean_columns(df):
    rename_map = {}
    for col in df.columns:
        new_col = col

        # Drop trailing "participated"
        new_col = re.sub(r"\s+participated$", "", new_col, flags=re.IGNORECASE)

        # Fix typo "LDTP" -> "LDP"
        new_col = new_col.replace("LDTP", "LDP")

        rename_map[col] = new_col
    
    return df.rename(columns=rename_map)

def process_df_with_combined_headers(df, location_name):
    """
    Process a dataframe with multi-level headers by combining column names with first row values.
    Creates column names like 'Beach Clean Up_Number of Beach Clean Ups'
    """
    # Make a copy to avoid modifying original
    df = df.copy()

    # Remove completely empty rows
    df = df.dropna(how='all').reset_index(drop=True)
    
    # Find the first row that has at least one non-null value
    first_valid_idx = df.first_valid_index()

    # Use that row as header
    condition=False
    if all(str(col).startswith("Unnamed") for col in df.columns):
        df.columns = df.iloc[first_valid_idx]
        df = df.iloc[first_valid_idx+1:].reset_index(drop=True)
        condition=True
    else:
        df = df.iloc[first_valid_idx:].reset_index(drop=True)

    if condition:
        main_headers=df.iloc[0].fillna('')
    else:
        # Get the main category headers (first row with data)
        main_headers = df.iloc[first_valid_idx].fillna('')

    # Fix column names by forward filling
    new_cols = []
    last_valid = None
    for col in df.columns:
        if ("Unnamed" in str(col)) or (str(col)=='nan'):
            new_cols.append(last_valid)   # reuse last valid name
        else:
            new_cols.append(col)          # set new name
            last_valid = col              # update last valid

    df.columns = new_cols

    df = df.dropna(how="all").reset_index(drop=True)
    
    new_columns = [str(i) + "_" + str(j) for i, j in zip(df.columns , main_headers)]
    df.columns = new_columns
    df = df.iloc[1:, :].reset_index(drop=True)
    
    if len(df) == 0:
        return pd.DataFrame()
    
    # Remove completely empty columns
    df = df.dropna(axis=1, how='all')
    
    # Add location column
    df['location'] = location_name
    
    # Remove rows containing 'total' (case-insensitive)
    if len(df) > 0:
        mask = df.astype(str).apply(lambda x: x.str.contains("total", case=False, na=False)).any(axis=1)
        df = df[~mask]
    
    # # # Remove rows that look like header repetitions
    # # if len(data_df) > 0:
    # #     # Check if any row has too many values that match column names
    # #     header_mask = pd.Series(False, index=data_df.index)
    # #     for idx, row in data_df.iterrows():
    # #         row_values = set(str(val).strip().lower() for val in row.values if pd.notna(val) and str(val).strip())
    # #         col_names = set(str(col).strip().lower() for col in data_df.columns)
            
    # #         # If more than 30% of non-null values match column names, it's likely a header row
    # #         if len(row_values) > 0:
    # #             matches = len(row_values.intersection(col_names))
    # #             if matches / len(row_values) > 0.3:
    # #                 header_mask[idx] = True
        
    #     data_df = data_df[~header_mask]
    
    return df.reset_index(drop=True)

In [382]:
name = "Erangal, Malad"
process_df_with_combined_headers(all_sheets[name], name)

,"Beach Name_Erangal, Malad",Beach Clean Up_Number of Beach Clean Ups,Beach Clean Up_Total Number of Participants,Beach Clean Up_Number of Beach Warrior Volunteers,Beach Clean Up_Number of Student Participants,Beach Clean Up_Number of Community Participants,Beach Clean Up_Number of RWA Participants,Beach Clean Up_Number of Government Representatives,Beach Clean Up_Number Local group volunteers,Beach Clean Up_Number of Academic Institutions,Beach Clean Up_Number of Residential Societies,Waste Management_Quantity of Marine Debris Collected (Kgs),"Waste Management_Quantity of Plastic (KGS)- Mix Plastic (Polythene, MLP, PVC, LDP)",Waste Management_Quantity of Cloth (KGS),"Waste Management_Quantity of Wood, Shoes, Bags and other materials (KGS)",location
0,18-19,26,3569,95,2919,64,72,120,299,2,1,6552,4258.8,982.8,1310.4,"Erangal, Malad"
1,19-20,52,4519,180,3291,153,162,260,473,5,2,14940,9711,2241,2988,"Erangal, Malad"


In [383]:
# Process all location tabs with the new function
print("Processing all location tabs with combined headers:")
print("=" * 60)

# Get all location tabs (exclude Sheet1)
location_tabs = [name for name in all_sheets.keys() if name != 'Sheet1']
processed_dataframes = []

for i, location_name in enumerate(location_tabs):
    print(f"\n{i+1:2d}. Processing: {location_name}")
    df = all_sheets[location_name]
    print(f"    Original shape: {df.shape}")
    
    try:
        processed_df = process_df_with_combined_headers(df, location_name)
        
        if not processed_df.empty:
            print(f"    Processed shape: {processed_df.shape}")
            print(f"    Columns: {len(processed_df.columns)} ({len(processed_df.columns)-1} data + 1 location)")
            processed_dataframes.append(processed_df)
        else:
            print(f"    ⚠️  WARNING: No data after processing")
            
    except Exception as e:
        print(f"    ❌ ERROR processing {location_name}: {str(e)}")

print(f"\n" + "=" * 60)
print(f"✅ Successfully processed: {len(processed_dataframes)}/{len(location_tabs)} location tabs")

# Show a sample of columns from different locations
if processed_dataframes:
    print(f"\nColumn comparison across locations:")
    all_columns = set()
    for df in processed_dataframes[:3]:  # Show first 3 for brevity
        location = df['location'].iloc[0]
        cols = [col for col in df.columns if col != 'location']
        all_columns.update(cols)
        print(f"\n  {location}: {len(cols)} columns")
        for col in cols[:5]:  # Show first 5 columns
            print(f"    • {col}")
        if len(cols) > 5:
            print(f"    ... and {len(cols)-5} more")
    
    print(f"\nTotal unique columns across all locations: {len(all_columns)}")

processed_dataframes[0] = processed_dataframes[0].drop('Overall Awareness Impact_', axis=1)
for idx, _ in enumerate(processed_dataframes):
    processed_dataframes[idx].columns = ["Period", *processed_dataframes[idx].columns[1:]]
    processed_dataframes[idx] = clean_columns(processed_dataframes[idx])
    processed_dataframes[idx] = processed_dataframes[idx].rename({"Period":'Year Period'}, axis=1)
    processed_dataframes[idx]['Year Period'] = clean_period_year(processed_dataframes[idx]['Year Period'])

Processing all location tabs with combined headers:

 1. Processing: Dadar
    Original shape: (11, 60)
    Processed shape: (9, 51)
    Columns: 51 (50 data + 1 location)

 2. Processing: Cuff Parade
    Original shape: (5, 16)
    Processed shape: (1, 16)
    Columns: 16 (15 data + 1 location)

 3. Processing: Chimbai Beach, Bandra
    Original shape: (5, 16)
    Processed shape: (2, 16)
    Columns: 16 (15 data + 1 location)

 4. Processing: Worli Fort, Worli
    Original shape: (5, 18)
    Processed shape: (3, 18)
    Columns: 18 (17 data + 1 location)

 5. Processing: Madh, Malad
    Original shape: (5, 16)
    Processed shape: (2, 16)
    Columns: 16 (15 data + 1 location)

 6. Processing: Erangal, Malad
    Original shape: (5, 16)
    Processed shape: (2, 16)
    Columns: 16 (15 data + 1 location)

 7. Processing: Gorai, Bhayander
    Original shape: (4, 16)
    Processed shape: (1, 16)
    Columns: 16 (15 data + 1 location)

 8. Processing: Prabhadevi Beach
    Original shape: 

In [384]:
def make_unique_columns(columns):
    """Make duplicate column names unique by appending suffixes."""
    seen = {}
    new_cols = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_cols.append(f"{col}_{seen[col]}")  # add suffix
        else:
            seen[col] = 0
            new_cols.append(col)
    return new_cols

all_cleaned = []

for df in processed_dataframes:
    name = df.location.unique()[0]
    temp = df.copy()

    # Fix duplicate column names
    temp.columns = make_unique_columns(temp.columns)

    # Add sheet name column
    temp["location"] = name

    all_cleaned.append(temp)

# Concatenate vertically, missing cols get NaN
final_df = pd.concat(all_cleaned, ignore_index=True, sort=False)

In [386]:
final_df.to_csv('../final_dataframe.csv', index=False)